<a href="https://colab.research.google.com/github/ShanthiniJoshitha/FOMC-statements-/blob/main/FinBert_final_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install NLTK
!pip install nltk

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords

In [ ]:
import pandas as pd

# Read CSV File
df = pd.read_csv('reports-in.csv')

# Extract Column into Tuple
links = tuple(df['links'])

print(links)


('https://www.federalreserve.gov/newsevents/pressreleases/monetary20060131a.htm', 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20060328a.htm', 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20060510a.htm', 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20060629a.htm', 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20060808a.htm', 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20060920a.htm', 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20061025a.htm', 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20061212a.htm', 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20070131a.htm', 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20070321a.htm', 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20070509a.htm', 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20070618a.htm', 'https://www.federalreserve

In [ ]:
import re
import pandas as pd


statement_dates = []
year = []

for link in links:
    match = re.search(r'\d+', link)
    if match:
        statement_dates.append(match.group())
        year.append(match.group()[:4])

data = {'year': year, 'statements_dates': statement_dates, 'links': links}

reports = pd.DataFrame(data)

# Convert columns to strings
reports = reports.applymap(str)

# Sort by statement_dates
reports = reports.sort_values(by='statements_dates')

print(reports)


     year statements_dates                                              links
0    2006         20060131  https://www.federalreserve.gov/newsevents/pres...
1    2006         20060328  https://www.federalreserve.gov/newsevents/pres...
2    2006         20060510  https://www.federalreserve.gov/newsevents/pres...
3    2006         20060629  https://www.federalreserve.gov/newsevents/pres...
4    2006         20060808  https://www.federalreserve.gov/newsevents/pres...
..    ...              ...                                                ...
146  2023         20230201  https://www.federalreserve.gov/newsevents/pres...
147  2023         20230322  https://www.federalreserve.gov/newsevents/pres...
148  2023         20230503  https://www.federalreserve.gov/newsevents/pres...
149  2023         20230614  https://www.federalreserve.gov/newsevents/pres...
150  2023         20230726  https://www.federalreserve.gov/newsevents/pres...

[151 rows x 3 columns]


In [ ]:
import re
import pandas as pd


statement_dates = []
year = []

for link in links:
    match = re.search(r'\d+', link)
    if match:
        statement_dates.append(match.group())
        year.append(match.group()[:4])

data = {'year': year, 'statements_dates': statement_dates, 'links': links}

reports = pd.DataFrame(data)

# Convert columns to strings
reports = reports.applymap(str)

# Sort by statement_dates
reports = reports.sort_values(by='statements_dates')

print(reports)


     year statements_dates                                              links
0    2006         20060131  https://www.federalreserve.gov/newsevents/pres...
1    2006         20060328  https://www.federalreserve.gov/newsevents/pres...
2    2006         20060510  https://www.federalreserve.gov/newsevents/pres...
3    2006         20060629  https://www.federalreserve.gov/newsevents/pres...
4    2006         20060808  https://www.federalreserve.gov/newsevents/pres...
..    ...              ...                                                ...
146  2023         20230201  https://www.federalreserve.gov/newsevents/pres...
147  2023         20230322  https://www.federalreserve.gov/newsevents/pres...
148  2023         20230503  https://www.federalreserve.gov/newsevents/pres...
149  2023         20230614  https://www.federalreserve.gov/newsevents/pres...
150  2023         20230726  https://www.federalreserve.gov/newsevents/pres...

[151 rows x 3 columns]


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Initialize lists to store scraped data
statement_content = []
statement_length = []

for link in links:
    # Send an HTTP GET request to the URL
    response = requests.get(link)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all <p> elements in the HTML
        paragraphs = soup.find_all('p')

        # Filter out irrelevant paragraphs (e.g., preliminary and last paragraphs)
        relevant_paragraphs = paragraphs[1:-1]  # Exclude the first and last paragraphs

        # Extract text content from the relevant paragraphs and join them
        statement_text = ' '.join([paragraph.get_text() for paragraph in relevant_paragraphs])

        # Remove line breaks
        statement_text = statement_text.replace('\n', ' ').strip()

        # Append the scraped content and its length to the respective lists
        statement_content.append(statement_text)
        statement_length.append(len(statement_text))

    else:
        print(f"Failed to retrieve content from {link}")

# Create a DataFrame to store the results
data = {'links': links, 'statement_content': statement_content, 'statement_length': statement_length}
reports_df = pd.DataFrame(data)

# Print the DataFrame
print(reports_df)


                                                 links  \
0    https://www.federalreserve.gov/newsevents/pres...   
1    https://www.federalreserve.gov/newsevents/pres...   
2    https://www.federalreserve.gov/newsevents/pres...   
3    https://www.federalreserve.gov/newsevents/pres...   
4    https://www.federalreserve.gov/newsevents/pres...   
..                                                 ...   
146  https://www.federalreserve.gov/newsevents/pres...   
147  https://www.federalreserve.gov/newsevents/pres...   
148  https://www.federalreserve.gov/newsevents/pres...   
149  https://www.federalreserve.gov/newsevents/pres...   
150  https://www.federalreserve.gov/newsevents/pres...   

                                     statement_content  statement_length  
0    Federal Open Market Committee   Monetary Polic...              2653  
1    Federal Open Market Committee   Monetary Polic...              2979  
2    Federal Open Market Committee   Monetary Polic...              3091  
3  

In [ ]:
i=0
for dates in reports['statements_dates']:
  reports_df['statements_dates'][i]=dates
  i=i+1
print(reports_df)

<ipython-input-11-6d857c3d1361>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reports_df['statements_dates'][i]=dates


                                                 links  \
0    https://www.federalreserve.gov/newsevents/pres...   
1    https://www.federalreserve.gov/newsevents/pres...   
2    https://www.federalreserve.gov/newsevents/pres...   
3    https://www.federalreserve.gov/newsevents/pres...   
4    https://www.federalreserve.gov/newsevents/pres...   
..                                                 ...   
146  https://www.federalreserve.gov/newsevents/pres...   
147  https://www.federalreserve.gov/newsevents/pres...   
148  https://www.federalreserve.gov/newsevents/pres...   
149  https://www.federalreserve.gov/newsevents/pres...   
150  https://www.federalreserve.gov/newsevents/pres...   

                                     statement_content  statement_length  \
0    Federal Open Market Committee   Monetary Polic...              2653   
1    Federal Open Market Committee   Monetary Polic...              2979   
2    Federal Open Market Committee   Monetary Polic...              3091   

In [ ]:
import pandas as pd

#reports = pd.DataFrame(data)

# Correct the date where 'statement_dates' is '20070618'
corrected_date = '20070628'
reports_df.loc[reports['statements_dates'] == '20070618', 'statements_dates'] = corrected_date

# Print the updated DataFrame
print(reports_df)


                                                 links  \
0    https://www.federalreserve.gov/newsevents/pres...   
1    https://www.federalreserve.gov/newsevents/pres...   
2    https://www.federalreserve.gov/newsevents/pres...   
3    https://www.federalreserve.gov/newsevents/pres...   
4    https://www.federalreserve.gov/newsevents/pres...   
..                                                 ...   
146  https://www.federalreserve.gov/newsevents/pres...   
147  https://www.federalreserve.gov/newsevents/pres...   
148  https://www.federalreserve.gov/newsevents/pres...   
149  https://www.federalreserve.gov/newsevents/pres...   
150  https://www.federalreserve.gov/newsevents/pres...   

                                     statement_content  statement_length  \
0    Federal Open Market Committee   Monetary Polic...              2653   
1    Federal Open Market Committee   Monetary Polic...              2979   
2    Federal Open Market Committee   Monetary Polic...              3091   

In [ ]:
def remove_text_after_voting(text):
    return re.sub(r'Voting for.*', '', text)

# Apply the function to the 'statement_content' column
reports_df['statement_content'] = reports_df['statement_content'].apply(remove_text_after_voting)

# Print the updated DataFrame
print(reports_df)





                                                 links  \
0    https://www.federalreserve.gov/newsevents/pres...   
1    https://www.federalreserve.gov/newsevents/pres...   
2    https://www.federalreserve.gov/newsevents/pres...   
3    https://www.federalreserve.gov/newsevents/pres...   
4    https://www.federalreserve.gov/newsevents/pres...   
..                                                 ...   
146  https://www.federalreserve.gov/newsevents/pres...   
147  https://www.federalreserve.gov/newsevents/pres...   
148  https://www.federalreserve.gov/newsevents/pres...   
149  https://www.federalreserve.gov/newsevents/pres...   
150  https://www.federalreserve.gov/newsevents/pres...   

                                     statement_content  statement_length  \
0    Federal Open Market Committee   Monetary Polic...              2653   
1    Federal Open Market Committee   Monetary Polic...              2979   
2    Federal Open Market Committee   Monetary Polic...              3091   

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

finbert = BertForSequenceClassification.from_pretrained('ahmedrachid/FinancialBERT-Sentiment-Analysis', num_labels=3)
tokenizer = BertTokenizer.from_pretrained('ahmedrachid/FinancialBERT-Sentiment-Analysis')


In [ ]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
from transformers import BertTokenizer
import nltk
from nltk.corpus import stopwords


# Create the sentiment analysis pipeline
#nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

In [ ]:
# Step 1: Load the finbert-tone model and set up tokenizer
model = BertForSequenceClassification.from_pretrained('ahmedrachid/FinancialBERT-Sentiment-Analysis', num_labels=3)
tokenizer = BertTokenizer.from_pretrained('ahmedrachid/FinancialBERT-Sentiment-Analysis')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
label_map = {0: "neutral", 1: "positive", 2: "negative"}

# Step 2: Define function for sentiment analysis
def analyze_sentiment(statement):
    inputs = tokenizer(statement, return_tensors="pt")
    inputs.to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    predicted_label = torch.argmax(outputs.logits).item()
    predicted_sentiment = label_map[predicted_label]

    return predicted_sentiment

# Step 3: Define function to process statements
def process_statements(row):
    lines = row['statement_content'].split('.')
    scores = [analyze_sentiment(line) for line in lines]

    positive_scores = (scores.count('positive') / len(scores))*100
    negative_scores = (scores.count('negative') / len(scores))*100
    neutral_scores = (scores.count('neutral') / len(scores))*100

    return pd.Series({
        'Year': row['statements_dates'][:4],
        'Date': pd.to_datetime(row['statements_dates'], format='%Y%m%d').strftime('%d-%m-%Y'),
        'Statement': row['statement_content'],
        'Mean Positive': positive_scores,
        'Mean Negative': negative_scores,
        'Mean Neutral': neutral_scores
    })

# Step 4: Apply the processing function to the reports_df DataFrame
results_df = reports_df.apply(process_statements, axis=1)

# Step 5: Save the results to an Excel file
results_df.to_excel('fomc_final_fr.xlsx', index=False)


In [ ]:
import pandas as pd

# Load the Excel file
df = pd.read_excel('fomc_final_fr.xlsx')

# Now 'df' contains the data from the Excel file
print(df)

     Year        Date                                          Statement  \
0    2006  31-01-2006  Federal Open Market Committee   Monetary Polic...   
1    2006  28-03-2006  Federal Open Market Committee   Monetary Polic...   
2    2006  10-05-2006  Federal Open Market Committee   Monetary Polic...   
3    2006  29-06-2006  Federal Open Market Committee   Monetary Polic...   
4    2006  08-08-2006  Federal Open Market Committee   Monetary Polic...   
..    ...         ...                                                ...   
146  2023  01-02-2023  Federal Open Market Committee   Monetary Polic...   
147  2023  22-03-2023  Federal Open Market Committee   Monetary Polic...   
148  2023  03-05-2023  Federal Open Market Committee   Monetary Polic...   
149  2023  14-06-2023  Federal Open Market Committee   Monetary Polic...   
150  2023  26-07-2023  Federal Open Market Committee   Monetary Polic...   

     Mean Positive  Mean Negative  Mean Neutral  
0        57.142857      28.571429    